In [44]:
import math
import numpy as np

from megengine import Tensor, tensor
import megengine.functional as F


def meshgrid(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    mesh_shape = (y.shape[0], x.shape[0])
    mesh_x = x.broadcast(mesh_shape)
    mesh_y = y.reshape(-1, 1).broadcast(mesh_shape)
    return mesh_x, mesh_y


def create_anchor_grid(featmap_size, offsets, stride):
    step_x, step_y = featmap_size
    shift = offsets * stride

    grid_x = F.arange(shift, step_x * stride + shift, step=stride)
    grid_y = F.arange(shift, step_y * stride + shift, step=stride)
    grids_x, grids_y = meshgrid(grid_y, grid_x)
    return grids_x.reshape(-1), grids_y.reshape(-1)


class AnchorGenerator:

    def __init__(
        self,
        anchor_scales: list = [[32], [64], [128], [256], [512]],
        anchor_ratios: list = [[0.5, 1, 2]],
        strides: list = [4, 8, 16, 32, 64],
        offset: float = 0,
    ):
        super().__init__()
        self.anchor_scales = np.array(anchor_scales, dtype=np.float32)
        self.anchor_ratios = np.array(anchor_ratios, dtype=np.float32)
        self.strides = strides
        self.offset = offset
        self.num_features = len(strides)

        self.base_anchors = self._different_level_anchors(anchor_scales, anchor_ratios)

    def _different_level_anchors(self, scales, ratios):
        if len(scales) == 1:
            scales *= self.num_features
        assert len(scales) == self.num_features

        if len(ratios) == 1:
            ratios *= self.num_features
        assert len(ratios) == self.num_features
        return [
            tensor(self.generate_base_anchors(scale, ratio))
            # self.generate_base_anchors(scale, ratio)
            for scale, ratio in zip(scales, ratios)
        ]

    def generate_base_anchors(self, scales, ratios):
        base_anchors = []
        areas = [s ** 2.0 for s in scales]
        for area in areas:
            for ratio in ratios:
                w = math.sqrt(area / ratio)
                h = ratio * w
                # center-based anchor
                x0, y0, x1, y1 = -w / 2.0, -h / 2.0, w / 2.0, h / 2.0
                base_anchors.append([x0, y0, x1, y1])
        return base_anchors

    def generate_anchors_by_features(self, sizes):
        all_anchors = []
        assert len(sizes) == self.num_features, (
            "input features expected {}, got {}".format(self.num_features, len(sizes))
        )
        for size, stride, base_anchor in zip(sizes, self.strides, self.base_anchors):
            grid_x, grid_y = create_anchor_grid(size, self.offset, stride)
            # FIXME: If F.stack works, change to stack
            grid_x, grid_y = grid_x.reshape(-1, 1), grid_y.reshape(-1, 1)
            grids = F.concat([grid_x, grid_y, grid_x, grid_y], axis=1)
            all_anchors.append(
                (grids.reshape(-1, 1, 4) + base_anchor.reshape(1, -1, 4)).reshape(-1, 4)
            )
        return all_anchors
    
    def __call__(self, featmaps):
        feat_sizes = [fmap.shape[-2:] for fmap in featmaps]
        return self.generate_anchors_by_features(feat_sizes)

    @property
    def anchor_dim(self):
        return 4

In [45]:
from megengine.random import gaussian
shape = [1, 3, 25, 30]
shape_list = reversed([shape[:2] + [s * 2**i for s in shape[2:]] for i in range(5)])
feature_maps = [gaussian(shape) for shape in shape_list]
for featmap in feature_maps:
    print("shape of feature map: {}".format(featmap.shape))

shape of feature map: (1, 3, 400, 480)
shape of feature map: (1, 3, 200, 240)
shape of feature map: (1, 3, 100, 120)
shape of feature map: (1, 3, 50, 60)
shape of feature map: (1, 3, 25, 30)


In [46]:
anchor_generator = AnchorGenerator()
anchors_list = anchor_generator(feature_maps)
for anchors, fmap in zip(anchors_list, feature_maps):
    print("anchor shape: {}".format(anchors.shape))
    print("{} = {} * {} * 3".format(anchors.shape[0], *fmap.shape[2:]))

anchor shape: (576000, 4)
576000 = 400 * 480 * 3
anchor shape: (144000, 4)
144000 = 200 * 240 * 3
anchor shape: (36000, 4)
36000 = 100 * 120 * 3
anchor shape: (9000, 4)
9000 = 50 * 60 * 3
anchor shape: (2250, 4)
2250 = 25 * 30 * 3


In [48]:
for anchors in anchors_list:
    print(anchors[anchors.shape[0] * 2 // 3 + 1])

Tensor([1264. 1048. 1296. 1080.])
Tensor([ 608. 1032.  672. 1096.])
Tensor([1216.  992. 1344. 1120.])
Tensor([ 512.  928.  768. 1184.])
Tensor([1024.  768. 1536. 1280.])
